In [1]:
from pyspark.sql import SparkSession
import os
import sys

import mlflow
from azure.storage.blob import BlobServiceClient

import importlib as importlib
#print("***curre dir",os.getcwd())
sys.path.append(os.path.abspath('../CNN-share-price-prediction/X-Channel-Images-Project'))
#print("##",sys.path)
from helper_functions_dir import credentials

In [2]:
_credentials = credentials.MLflow_Credentials()
_credentials.get_credentials()

_credentials
conn_str = _credentials.AZURE_STORAGE_CONNECTION_STRING


In [19]:
# Set the environment variables for PySpark
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Azure SQL Database Query") \
    .config("spark.driver.extraClassPath", _credentials.sqljdb_connector_location) \
    .getOrCreate()

# Define the JDBC URL and connection properties
jdbc_url = f"jdbc:sqlserver://{_credentials.server_db}.database.windows.net:{_credentials.port_db};database={_credentials.mlflow_db}"
jdbc_properties = {
    "user": _credentials.username_db,
    "password": _credentials.password_db,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [20]:
#delete blobs for run_uuis of those experiments deleted

#delete blobs
container_name = _credentials.storage_container_name
directory_name= _credentials.blob_directory

experiments_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "dbo.experiments") \
    .option("user", jdbc_properties["user"]) \
    .option("password", jdbc_properties["password"]) \
    .option("driver", jdbc_properties["driver"]) \
    .load()

deleted_experiments_df = experiments_df.filter(experiments_df.lifecycle_stage == 'deleted')
deleted_experiment_ids_df = deleted_experiments_df.select("experiment_id")

runs_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "dbo.runs") \
    .option("user", jdbc_properties["user"]) \
    .option("password", jdbc_properties["password"]) \
    .option("driver", jdbc_properties["driver"]) \
    .load()

deleted_experiments_runs_df = runs_df.join(deleted_experiment_ids_df,'experiment_id').filter(runs_df.lifecycle_stage == 'deleted')

print("deleted_experiments_runs_df.count()",deleted_experiments_runs_df.count())
print(deleted_experiments_runs_df.show(n=deleted_experiments_runs_df.count(), truncate=False))

blob_service_client = BlobServiceClient.from_connection_string(conn_str)
    
# List all blobs in the container under the specified directory
container_client = blob_service_client.get_container_client(container_name)

run_uuids = [row.run_uuid for row in deleted_experiments_runs_df.select("run_uuid").collect()]

for run_uuid in run_uuids:
    print("run_uuid",run_uuid)
    blobs = container_client.list_blobs(name_starts_with=directory_name)
    for blob in blobs:
        print("blob",blob.name)
        if run_uuid in blob.name:
            blob_client = container_client.get_blob_client(blob)
            print("delete blob",blob.name)
            blob_client.delete_blob()

deleted_experiments_runs_df.count() 59
+-------------+--------------------------------+------+-----------+-----------+----------------+-------+--------+-------------+-------------+--------------+---------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+
|experiment_id|run_uuid                        |name  |source_type|source_name|entry_point_name|user_id|status  |start_time   |end_time     |source_version|lifecycle_stage|artifact_uri                                                                                                                                                                                          |deleted_time |
+-------------+--------------------------------+------+-----------+-----------+----------------+-------+--------+-------------+-------------+--------------+---------------+-----------------

In [21]:
#delete blobs for run_uuis deleted

runs_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "dbo.runs") \
    .option("user", jdbc_properties["user"]) \
    .option("password", jdbc_properties["password"]) \
    .option("driver", jdbc_properties["driver"]) \
    .load()

deleted_run_uuids_df = runs_df.filter(runs_df.lifecycle_stage == 'deleted')

print("deleted_runs_df.count()",deleted_run_uuids_df.count())
print(deleted_run_uuids_df.show(n=deleted_run_uuids_df.count(), truncate=False))

blob_service_client = BlobServiceClient.from_connection_string(conn_str)
    
# List all blobs in the container under the specified directory
container_client = blob_service_client.get_container_client(container_name)

run_uuids = [row.run_uuid for row in deleted_run_uuids_df.select("run_uuid").collect()]

print("runs",run_uuids)
for run_uuid in run_uuids:
    print("run_uuid",run_uuid)
    blobs = container_client.list_blobs(name_starts_with=directory_name)
    for blob in blobs:
        print("possible blob to delete:",blob.name)
        if run_uuid in blob.name:
            blob_client = container_client.get_blob_client(blob)
            print("delete blob",blob.name)
            blob_client.delete_blob()

deleted_runs_df.count() 59
+--------------------------------+------+-----------+-----------+----------------+-------+--------+-------------+-------------+--------------+---------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+-------------+
|run_uuid                        |name  |source_type|source_name|entry_point_name|user_id|status  |start_time   |end_time     |source_version|lifecycle_stage|artifact_uri                                                                                                                                                                                          |experiment_id|deleted_time |
+--------------------------------+------+-----------+-----------+----------------+-------+--------+-------------+-------------+--------------+---------------+-------------------------------------------

In [22]:
# delete residual tables after experiment or run_uuid deleted via ui
    
experiments_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "dbo.experiments") \
    .option("user", jdbc_properties["user"]) \
    .option("password", jdbc_properties["password"]) \
    .option("driver", jdbc_properties["driver"]) \
    .load()

experiment_tags_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "dbo.experiment_tags") \
    .option("user", jdbc_properties["user"]) \
    .option("password", jdbc_properties["password"]) \
    .option("driver", jdbc_properties["driver"]) \
    .load()

latest_metrics_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "dbo.latest_metrics") \
    .option("user", jdbc_properties["user"]) \
    .option("password", jdbc_properties["password"]) \
    .option("driver", jdbc_properties["driver"]) \
    .load()

metrics_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "dbo.metrics") \
    .option("user", jdbc_properties["user"]) \
    .option("password", jdbc_properties["password"]) \
    .option("driver", jdbc_properties["driver"]) \
    .load()

tags_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "dbo.tags") \
    .option("user", jdbc_properties["user"]) \
    .option("password", jdbc_properties["password"]) \
    .option("driver", jdbc_properties["driver"]) \
    .load()

params_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "dbo.params") \
    .option("user", jdbc_properties["user"]) \
    .option("password", jdbc_properties["password"]) \
    .option("driver", jdbc_properties["driver"]) \
    .load()

runs_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "dbo.runs") \
    .option("user", jdbc_properties["user"]) \
    .option("password", jdbc_properties["password"]) \
    .option("driver", jdbc_properties["driver"]) \
    .load()

datasets_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "dbo.datasets") \
    .option("user", jdbc_properties["user"]) \
    .option("password", jdbc_properties["password"]) \
    .option("driver", jdbc_properties["driver"]) \
    .load()

In [23]:
#remove experiment ids deleted from experiment_tags
deleted_experiment_ids = experiments_df.filter(experiments_df.lifecycle_stage == 'deleted').select('experiment_id')

filtered_experiment_tags_to_keep_df = experiment_tags_df.join(deleted_experiment_ids, on='experiment_id', how='left_anti')

print("filtered_experiment_tags_to_keep_df")
filtered_experiment_tags_to_keep_df.show()

print("Pre experiment_tags_df")
experiment_tags_df.show()

filtered_experiment_tags_to_keep_df.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "dbo.experiment_tags") \
    .option("user", jdbc_properties["user"]) \
    .option("password", jdbc_properties["password"]) \
    .option("driver", jdbc_properties["driver"]) \
    .mode("overwrite") \
    .save()

print("Post experiment_tags_df")
experiment_tags_df.show()

filtered_experiment_tags_to_keep_df
+-------------+---+-----+
|experiment_id|key|value|
+-------------+---+-----+
+-------------+---+-----+

Pre experiment_tags_df
+-------------+---+-----+
|experiment_id|key|value|
+-------------+---+-----+
+-------------+---+-----+

Post experiment_tags_df
+-------------+---+-----+
|experiment_id|key|value|
+-------------+---+-----+
+-------------+---+-----+



In [25]:
#delete experiment ids from latest_metrics
print("latest_metrics_df")
latest_metrics_df.show()
filtered_latest_metrics_to_keep_df = latest_metrics_df.join(runs_df.join(deleted_experiment_ids, on='experiment_id').select('experiment_id'), on='experiment_id', how='left_anti')

print("filtered_latest_metrics_to_keep_df")
filtered_latest_metrics_to_keep_df.show()

print("Pre latest_metrics_df")
latest_metrics_df.show()

filtered_experiment_tags_to_keep_df.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "dbo.latest_metrics") \
    .option("user", jdbc_properties["user"]) \
    .option("password", jdbc_properties["password"]) \
    .option("driver", jdbc_properties["driver"]) \
    .mode("overwrite") \
    .save()

print("Post latest_metrics_df")
latest_metrics_df.show()

latest_metrics_df
+-------------+---+-----+
|experiment_id|key|value|
+-------------+---+-----+
+-------------+---+-----+

filtered_latest_metrics_to_keep_df
+-------------+---+-----+
|experiment_id|key|value|
+-------------+---+-----+
+-------------+---+-----+

Pre latest_metrics_df
+-------------+---+-----+
|experiment_id|key|value|
+-------------+---+-----+
+-------------+---+-----+

Post latest_metrics_df
+-------------+---+-----+
|experiment_id|key|value|
+-------------+---+-----+
+-------------+---+-----+



In [ ]:
#todo
filtered_metrics_df = metrics_df.join(runs_df.join(deleted_experiment_ids, on='experiment_id').select('run_uuid'), on='run_uuid', how='left_anti')
filtered_metrics_df.write.format("jdbc").options(
    url="jdbc:your_database_url",
    dbtable="dbo.metrics",
    user="your_username",
    password="your_password",
    mode="overwrite"
).save()


In [ ]:
#tiodo
filtered_tags_df = tags_df.join(runs_df.join(deleted_experiment_ids, on='experiment_id').select('run_uuid'), on='run_uuid', how='left_anti')
filtered_tags_df.write.format("jdbc").options(
    url="jdbc:your_database_url",
    dbtable="dbo.tags",
    user="your_username",
    password="your_password",
    mode="overwrite"
).save()


In [ ]:
#todo
filtered_params_df = params_df.join(runs_df.join(deleted_experiment_ids, on='experiment_id').select('run_uuid'), on='run_uuid', how='left_anti')
filtered_params_df.write.format("jdbc").options(
    url="jdbc:your_database_url",
    dbtable="dbo.params",
    user="your_username",
    password="your_password",
    mode="overwrite"
).save()


In [ ]:
#todo
filtered_runs_df = runs_df.join(deleted_experiment_ids, on='experiment_id', how='left_anti')
filtered_runs_df.write.format("jdbc").options(
    url="jdbc:your_database_url",
    dbtable="dbo.runs",
    user="your_username",
    password="your_password",
    mode="overwrite"
).save()


In [ ]:
#todo
filtered_experiments_df = experiments_df.filter(experiments_df.lifecycle_stage != 'deleted')
filtered_experiments_df.write.format("jdbc").options(
    url="jdbc:your_database_url",
    dbtable="dbo.experiments",
    user="your_username",
    password="your_password",
    mode="overwrite"
).save()


In [ ]:
#todo
filtered_datasets_df = datasets_df.join(deleted_experiment_ids, on='experiment_id', how='left_anti')
filtered_datasets_df.write.format("jdbc").options(
    url="jdbc:your_database_url",
    dbtable="datasets",
    user="your_username",
    password="your_password",
    mode="overwrite"
).save()


In [ ]:
spark.stop()